# Deployment of the model 



## Connect to the client 

In [1]:
from lib import get_client
client=get_client()
print(f'workspace name: {client.workspace_name}')
datastore=client.datastores.get_default()
print(f'datastore name: {datastore.name} type: {datastore.type}')

workspace name: azure-ml
datastore name: storecapture type: DatastoreType.AZURE_DATA_LAKE_GEN2


## Get the Model 

In [35]:
model_name = "grstb-signal-classification"
registered_model=client.models.get(name=model_name,version='1')
print(f'registered model id: {registered_model.id}')

registered model id: /subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourceGroups/azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml/models/grstb-signal-classification/versions/1


## Create an Online endpoint

In [42]:

from datetime import datetime
from azure.ai.ml.entities import ManagedOnlineEndpoint


online_endpoint_name = "grstb" + datetime.now().strftime(
    "%m%d%H%M%f"
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Grstb signal recognition endpoint",
    auth_mode="key",
)
print(f'endpoint name: {online_endpoint_name}')

grstb05141353496629


In [43]:
client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://grstb05141353496629.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://grstb05141353496629.eastus.inference.ml.azure.com/swagger.json', 'name': 'grstb05141353496629', 'description': 'Grstb signal recognition endpoint', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourcegroups/azure-ml/providers/microsoft.machinelearningservices/workspaces/azure-ml/onlineendpoints/grstb05141353496629', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:284c1630-7a4b-4b08-93d2-12e6c09f7fe2:b79a269b-7f4b-45df-aa1e-c394a2fcb4d0?api-version=2022-02-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourceGroups/azure-ml/providers/Mi

In [47]:
endpoints=client.online_endpoints.get(online_endpoint_name)
print(f'Name: {endpoints.name}')
print(f'Swagger URL: {endpoints.openapi_uri}')
print(f'inference URL: {endpoints.scoring_uri}')

Name: grstb05141353496629
Swagger URL: https://grstb05141353496629.eastus.inference.ml.azure.com/swagger.json
inference URL: https://grstb05141353496629.eastus.inference.ml.azure.com/score


## Create a deployment for the online point

In [48]:
from azure.ai.ml.entities import ManagedOnlineDeployment, ProbeSettings
deployment = ManagedOnlineDeployment(
    name="grstb-mlflow-deploy",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_E2s_v3",
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

In [49]:
client.online_deployments.begin_create_or_update(deployment).result()

Check: endpoint grstb05141353496629 exists


............................................................................................................................................................................

HttpResponseError: (None) OutOfQuota: Unable to download user model, not enough space on the disk. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-outofquota
Code: None
Message: OutOfQuota: Unable to download user model, not enough space on the disk. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-outofquota
Exception Details:	(None) OutOfQuota: Unable to download user model, not enough space on the disk. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-outofquota
	Code: None
	Message: OutOfQuota: Unable to download user model, not enough space on the disk. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-outofquota

In [ ]:
deployment=client.online_deployments.get(deployment.name)
print(deployment)